This Notebook wil be mainly used for the Capstone Project (III)

    Import python pandas library to import data as dataframe

In [1]:
#import python pandas library
import pandas as pd

    Import python numpy mathematical library

In [2]:
#import python numpy library
import numpy as np

    Import requests library to get data from websites URL

In [3]:
#import requests library
import requests

    Import BeatifulSoup webscrapping library

In [4]:
#import BeatifulSoup library
from bs4 import BeautifulSoup

    URL contains Wikipedia website with Toronto postal codes, boroughs and neighborhoods

In [5]:
#URL contains Wikipedia website

In [6]:
URL= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

    Request website page from website URL

In [7]:
#Request website page

In [8]:
page= requests.get(URL)

    Create Beautiful Soup object soup that takes the HTML content you scraped earlier (page)
    as its input

In [9]:
#Create Beautiful Soup object soup that takes the HTML content you scraped earlier (page) as its input

In [10]:
soup= BeautifulSoup(page.content, "html.parser")

    Find all cells in tables

In [11]:
#Find all cells
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    #If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
    if row.span.text=='Not assigned':
        pass
    #If more than one neighborhood exist in one postal code area, then combine rows into one row with the neighborhoods separated with a comma
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

    Table_contents as a Pandas Dataframe

In [12]:
#table_contents as a Pandas Dataframe
df=pd.DataFrame(table_contents)

#Shortening mail distribution centers names
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})



In [13]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


    Read Geospatial Coordinates file

In [14]:
#Read Geospatial Coordinates
geo_data= "https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"

    Importing Geospatial Coordinates csv file to data frame df_geo_data

In [15]:
#Importing Geospatial Coordinates csv file
df_geo_data= pd.read_csv(geo_data)
df_geo_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


    Lookup coordinates for each postal code in dataframe df through loop in dataframe
    df_geo_data

In [16]:
#Lookup coordinates for each postal code in dataframe df
df_geo= pd.DataFrame(columns= ["Latitude","Longitude"])

print(df_geo)

#Total number of rows in Dataframe df
total_rows= len(df)

#Counter for appending rows to new dataframe df_geo with same row order than dataframe df
rows= 0

#For loop to get Geospatial Coordinates from a Postal Code
for row in range(total_rows):
    for row_geo in range(total_rows):
        if df.iloc[row,0] == df_geo_data.iloc[row_geo,0] and rows != total_rows:
            #df_geo= df_geo.append({"Latitude": df_geo_data.iloc[rows, 0], "Longitude": df_geo_data.iloc[rows, 1]}, ignore_index= True)
            rows= rows +1
            df_geo= df_geo.append({"Latitude": df_geo_data.iloc[row_geo, 1], "Longitude": df_geo_data.iloc[row_geo, 2]}, ignore_index= True)               
            
df_geo



Empty DataFrame
Columns: [Latitude, Longitude]
Index: []


,Latitude,Longitude
0,43.753259,-79.329656
1,43.725882,-79.315572
2,43.654260,-79.360636
3,43.718518,-79.464763
4,43.662301,-79.389494
...,...,...
98,43.653654,-79.506944
99,43.665860,-79.383160
100,43.662744,-79.321558
101,43.636258,-79.498509


    Merge data of dataframe df and Geospatial Coordinates dataframe df_geo

In [17]:
#Merge data of dataframe df and dataframe df_geo
df=pd.concat([df, df_geo], axis=1)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
